In [ ]:
#Data getting fed to LSTM
all_signals_list = ["body_acc_x_", "body_acc_y_", "body_acc_z_", "body_gyro_x_", "body_gyro_y_", "body_gyro_z_", 
                   "total_acc_x_", "total_acc_y_", "total_acc_z_"]

In [ ]:
#Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
warnings.simplefilter("ignore")

In [ ]:
#Function used to read each signal data file
def reading_data(filename):
    return pd.read_csv(filename, delim_whitespace = True, header = None)

In [ ]:
#Combine test/train data from each signal into matrix form
def total_signal_matrix(trainOrTest):
    complete_data = []
    for signal in all_signals_list:
        data = reading_data("../content/drive/MyDrive/HARData/"+ trainOrTest +"/Inertial Signals/"+ signal + trainOrTest +".txt")
        complete_data.append(data)
    return np.transpose(complete_data, (1, 2, 0))

In [ ]:
#Get activity label and store in df (6 wide in this case for 6 activites)
def load_labels(subset):
    filename = "../content/drive/MyDrive/HARData/"+subset+"/y_"+subset+".txt"
    y = reading_data(filename)
    return pd.get_dummies(y[0])
# here, get_dummies takes pandas series as input and returns its one-hot encoded vector of each element in a series.

In [ ]:
#Load test/train data and labels
def load_full_data():
    x_train = total_signal_matrix("train")
    y_train = load_labels("train")
    x_test = total_signal_matrix("test")
    y_test = load_labels("test")
    return x_train, y_train, x_test, y_test

In [ ]:
#Display training results
def print_confusionMatrix(Y_TestLabels, PredictedLabels):
    confusionMatx = confusion_matrix(Y_TestLabels, PredictedLabels)
    
    precision = confusionMatx/confusionMatx.sum(axis = 0)
    
    recall = (confusionMatx.T/confusionMatx.sum(axis = 1)).T
    
    sns.set(font_scale=1.5)
    
    labels = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LYING"]
    
    plt.figure(figsize=(16,7))
    sns.heatmap(confusionMatx, cmap = "Blues", annot = True, fmt = ".1f", xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 20)
    plt.ylabel('Original Class', fontsize = 20)
    plt.tick_params(labelsize = 15)
    plt.xticks(rotation = 90)
    plt.show()
    
    print("-"*125)
    
    plt.figure(figsize=(16,7))
    sns.heatmap(precision, cmap = "Blues", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("Precision Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 20)
    plt.ylabel('Original Class', fontsize = 20)
    plt.tick_params(labelsize = 15)
    plt.xticks(rotation = 90)
    plt.show()
    
    print("-"*125)
    
    plt.figure(figsize=(16,7))
    sns.heatmap(recall, cmap = "Blues", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("Recall Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 20)
    plt.ylabel('Original Class', fontsize = 20)
    plt.tick_params(labelsize = 15)
    plt.xticks(rotation = 90)
    plt.show()

In [ ]:
#Store dfs and print sizes
x_train, y_train, x_test, y_test = load_full_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

NameError: ignored

In [ ]:
#Save data (not necessary)
np.save("../content/drive/MyDrive/HARData/train", x_train)
np.save("../content/drive/MyDrive/HARData/train_label", y_train)
np.save("../content/drive/MyDrive/HARData/test", x_test)
np.save("../content/drive/MyDrive/HARData/test_label", y_test)

In [ ]:
#Load data in numpy format
def data():
    x_train = np.load("../content/drive/MyDrive/HARData/train.npy")
    y_train = np.load("../content/drive/MyDrive/HARData/train_label.npy")
    x_test = np.load("../content/drive/MyDrive/HARData/test.npy")
    y_test = np.load("../content/drive/MyDrive/HARData/test_label.npy")
    return x_train, y_train, x_test, y_test

In [ ]:
#Number of classes (in this case types of human activity)
def count_unique_classes(y_train):
    return len(set([tuple(a) for a in y_train]))

In [ ]:
#Builing ml model
def create_model(x_train, y_train, x_test, y_test):
    
    epochs = 8 #iterate through dataset 8 times
    batch_size = 32 #default batch size
    timesteps = x_train.shape[1] #how many data points for each input signal per timestep
    input_dim = len(x_train[0][0]) #number of input signals
    n_classes = 6 #number of classes (activities)
    
    model = Sequential() #keras sequential
    
    model.add(LSTM(64, return_sequences = True, input_shape = (timesteps, input_dim))) #lstm layer 64 unit output space, return all sequences
    model.add(Dropout({{uniform(0, 1)}})) #hyper-parameter optimization using hyperas 
    
    model.add(LSTM({{choice([32, 16])}})) #not too sure about this line
    model.add(Dropout({{uniform(0, 1)}})) #hyper-parameter optimization using hyperas (dropout layer helps prevent overfitting)
    
    model.add(Dense(n_classes, activation='sigmoid')) #dense layer with output as activities, using sigmoid activation function
    
    print(model.summary())
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop') #compile with rmsprop optimizer, crossentropy loss function and an accuracy metric
    
    result = model.fit(x_train, y_train, batch_size = batch_size, epochs=epochs, verbose=2, validation_split=0.01) #train model (8 epochs, 32 batch-size, verbose = 2 for one output print per epoch, validation split small because we have test set already)
    
    validation_acc = np.amax(result.history['val_accuracy']) #model accuracy 
    
    print('Best validation acc of epoch:', validation_acc)
    
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=create_model, data=data, algo=tpe.suggest, max_evals=4, trials=Trials(), notebook_name = "/drive/MyDrive/Colab Notebooks/QmindHAR") #Use hyperas to for hyper parameterization of model
x_train, y_train, x_test, y_test = data()

score = best_model.evaluate(x_test, y_test) #evaluate best model

print('---------------------')
print('|      Accuracy      |') #print results
print('---------------------')
acc = np.round((score[1]*100), 2)
print(str(acc)+"%\n")
    
print('----------------------------------')
print('|      Best Hyper-Parameters      |') # print best hyper parameters
print('----------------------------------')
print(best_run)
print("\n\n")

true_labels = [np.argmax(i)+1 for i in y_test]
predicted_probs = best_model.predict(x_test)
predicted_labels = [np.argmax(i)+1 for i in predicted_probs]
print_confusionMatrix(true_labels, predicted_labels) #display results

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.manifold import TSNE
except:
    pass

try:
    import warnings
except:
    pass

try:
    from datetime import datetime
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from sklearn.metrics import confusion_matrix
except:
    pass

try:
    from sklearn.metrics import accuracy_score
except:
    pass

try:
    from sklearn.linear_model import LogisticRegression
except:
    pass

try:
    from sklearn.svm import LinearSVC
except:
    pass

try:
    from sklearn.svm import SVC
except:
    pass

try:
    from sklearn.tree import DecisionTreeClassifier
except:
    pass

try:
    from sklearn.ensemble import RandomForestClassifier
except:
    pass

try:
    from sklearn.ensemble import Gradi

NameError: ignored